# Newton's Complete Method

The system tests mentioned above will start. Initially, the data created using the previous code and the system data, found in [IEEE14](https://labs.ece.uw.edu/pstca/pf14/pg_tca14bus.htm), will be used.

We started the process with the initialization of the libraries

In [ ]:
import math
import numpy as np
from numpy.linalg import inv
import time
import matplotlib.pyplot as plt

Database identification and remove unnecessary lines

In [ ]:
branch_txt = '../Data/System_IEEE_14bus_branch.txt'    
node_txt = '../Data/System_IEEE_14bus_node.txt'    
node2_txt = '../Data/Data_Node.txt' 

data_branches = np.loadtxt(branch_txt, skiprows = 2)
node_data = np.loadtxt(node_txt, skiprows = 3)
node2_data = np.loadtxt(node2_txt, skiprows = 1)

In this part of the code the database data is separated into parameters, for the tests an apparent power of $S^{base}=100 MVA$  was used and an error factor $\zeta = 10^{-5}$

In [ ]:
S_base = 100                                  
zeta = 10**-5         

start_time = time.time()               

if np.shape(data_branches) == (7,):
    data_branches = data_branches.reshape((1, 7))

TO =    data_branches[:,0]      
FROM =  data_branches[:,1]      
r =     data_branches[:,2]      
x =     data_branches[:,3]      
bsh =   data_branches[:,4]/2      
a =     data_branches[:,5]      
phi =   data_branches[:,6]

NN = max(node_data[:,0])      
NN = int(NN)

bus = node_data[:,0]          
Type = node_data[:,1]   

V = node_data[:,2]            
theta = node_data[:,3]

P_G = node_data[:,4]/S_base    
Q_G = node_data[:,5]/S_base    
B_shunt = node_data[:,6]/S_base 

P_C = node2_data[:,1]/S_base    
Q_C = node2_data[:,2]/S_base    

P_nom = P_G - P_C               
Q_nom = Q_G - Q_C    

NN = int(max(max(TO),max(FROM)))    
NB = len(TO)             

Calculation of line parameters, such as admittance, shunting capacitors and calculation of matrix G and B.

In [ ]:
Y = np.zeros([NN,NN], dtype=complex)
tii = np.zeros([NB])
tjj = np.zeros([NB])
tij = np.zeros([NB])

for b in range(NN):  
    Y[b][b] = B_shunt[b] *1j;

for b in range(NB):
    i = int(TO[b])-1
    j = int(FROM[b])-1   
    
    ykm = 1/(r[b]+x[b]*1j)
 
    tii[b] = 1/(a[b]*a[b])
    tjj[b] = 1
    tij[b] = 1/ a[b]

    Y[i][i] = Y[i][i] + tii[b]*ykm+ 1j*bsh[b]
    Y[j][j] = Y[j][j]+ tjj[b]*ykm+ 1j*bsh[b]
    

    Y[i][j] = Y[i][j] - tij[b]*ykm
    Y[j][i] = Y[j][i] - tij[b]*ykm

G = np.real(Y)                
B = np.imag(Y)              

The iterative process is started

In [ ]:
itera = 0 
maxDP = 10**5
maxDQ = 10**5

mismP = np.array([])
mismQ = np.array([])

while (abs(maxDP) > zeta or abs(maxDQ) > zeta):    

    P_Calc = np.zeros_like(P_nom)
    Q_Calc = np.zeros_like(Q_nom)
    for i in range(NN):
        P_Calc[i] = G[i][i] * V[i] * V[i]
        Q_Calc[i] =-B[i][i] * V[i] * V[i]
    
    for w in range(NB):    
        i = int(TO[w])-1
        j = int(FROM[w])-1        
    
        ykm = 1/(r[w]+x[w]*1j)
    
        g = np.real(ykm) * tij[w]
        b = np.imag(ykm) * tij[w]
    
        ab = theta[i]-theta[j]+ phi[w]    

        P_Calc[i] = P_Calc[i] + V[i] * V[j] * ( - g * math.cos(ab) - b * math.sin(ab))
        P_Calc[j] = P_Calc[j] + V[i] * V[j] * ( - g * math.cos(ab) + b * math.sin(ab))
        
        Q_Calc[i] = Q_Calc[i] + V[i] * V[j] * ( - g * math.sin(ab) + b * math.cos(ab))
        Q_Calc[j] = Q_Calc[j] - V[i] * V[j] * ( - g * math.sin(ab) - b * math.cos(ab))

    DP = np.zeros_like(P_nom)    
    DQ = np.zeros_like(Q_nom)
    
    maxDP = 0
    maxDQ = 0    
    
    for i in range(NN):        
        if Type[i]!= 3:
            DP[i] = P_nom[i] - P_Calc[i]
            
            if abs(DP[i]) > abs(maxDP):
                maxDP = DP[i]
                
        if Type[i]<= 1:
            DQ[i] = Q_nom[i] - Q_Calc[i]
            
            if abs(DQ[i]) > abs(maxDQ):
                maxDQ = DQ[i]    

    mismQ = np.append(mismQ, maxDQ)
    mismP = np.append(mismP, maxDP)

    DS = np.r_[DP,DQ]    
    
    H = np.zeros((NN, NN))
    N = np.zeros((NN, NN))
    M = np.zeros((NN, NN))
    L = np.zeros((NN, NN))
    
    for i in range(NN):
        
        H[i][i] =-Q_Calc[i] - V[i]**2 * B[i][i]
        if Type[i] == 3:
            H[i][i] = 10**10     
        N[i][i] = P_Calc[i] + V[i] * G[i][i]
        M[i][i] = P_Calc[i] - V[i]* V[i] * G[i][i]
        L[i][i] = Q_Calc[i] - V[i] * B[i][i]        
        if Type[i] >= 2:
            L[i][i] = 10**10
    
    for w in range(NB):    
        i = int(TO[w])-1
        j = int(FROM[w])-1        
        
        ab = theta[i]-theta[j]-(phi[w]*math.pi/180)
        
        H[i][j] = V[i] * V[j] * ( G[i][j] * math.sin(ab) - B[i][j] * math.cos(ab))
        H[j][i] = V[i] * V[j] * (-G[i][j] * math.sin(ab) - B[i][j] * math.cos(ab))
    
        N[i][j] = V[i] * (G[i][j] * math.cos(ab) + B[i][j] * math.sin(ab))
        N[j][i] = V[j] * (G[i][j] * math.cos(ab) - B[i][j] * math.sin(ab))
        
        M[i][j] =-V[i] * V[j] * (G[i][j] * math.cos(ab) + B[i][j] * math.sin(ab))
        M[j][i] =-V[i] * V[j] * (G[i][j] * math.cos(ab) - B[i][j] * math.sin(ab))
    
        L[i][j] = V[i] * (G[i][j] * math.sin(ab) - B[i][j] * math.cos(ab))
        L[j][i] = V[j] * (G[i][j] * math.sin(ab) - B[i][j] * math.cos(ab))    
    
    J1 = np.hstack((H, N))
    J2 = np.hstack((M, L))
    J = np.linalg.inv(np.vstack((J1, J2)))

    DV = np.dot(J,DS)
    
    d_theta = DV[:NN]
    dV = DV[NN:]

    theta = d_theta + theta        
    V = dV + V    

    itera += 1
    if itera >= 200:
        print("Maximum number of iterations reached.")
        break    

total_time = time.time() - start_time

Graph of power mismatches, along the iterative process

In [ ]:
x = range(itera)
y1= abs(mismQ)
y2= abs(mismP)
plt.ylabel('Power mismatches')
plt.xlabel('Iterations')

plt.ylim(ymax = 1, ymin = 0)
plt.xlim(xmax = itera, xmin = 0)
plt.plot(x, y1, y2)
plt.show()

Graph of the voltage  profile in each of the bars

In [ ]:
x = range(NN)
y= abs(V)
plt.ylabel('Magnitude of tension [p.u]')
plt.xlabel('Node')

plt.ylim(ymax = 1.2, ymin = 0)
plt.xlim(xmax = 13, xmin = 0)
plt.plot(x, y)
plt.show()

# Newton Fast Decoupled
In the second part of the project, the power flow will be calculated using the Newton Fast Decoupled methodology. The process will start from the creation and collection of the database, for the parameters.

In [ ]:
S_base = 100                                  
zeta = 10**-5         

start_time = time.time()               

if np.shape(data_branches) == (7,):
    data_branches = data_branches.reshape((1, 7))

TO =    data_branches[:,0]      
FROM =  data_branches[:,1]      
r =     data_branches[:,2]      
x =     data_branches[:,3]      
bsh =   data_branches[:,4]/2      
a =     data_branches[:,5]      
phi =   data_branches[:,6]

NN = max(node_data[:,0])      
NN = int(NN)

bus = node_data[:,0]          
Type = node_data[:,1]   

V = node_data[:,2]            
theta = node_data[:,3]

P_G = node_data[:,4]/S_base    
Q_G = node_data[:,5]/S_base    
B_shunt = node_data[:,6]/S_base 

P_C = node2_data[:,1]/S_base    
Q_C = node2_data[:,2]/S_base    

P_nom = P_G - P_C               
Q_nom = Q_G - Q_C    

NN = int(max(max(TO),max(FROM)))    
NB = len(TO)       

Performing the calculation of the line parameters, as in the previous method, but adding the matrices $ B ^{\prime}$ and $ B^{\prime\prime} $.

In [ ]:
Y = np.zeros([NN,NN], dtype=complex)
tii = np.zeros([NB])
tjj = np.zeros([NB])
tij = np.zeros([NB])
tij = np.zeros([NB])

for b in range(NN):  
    Y[b][b] = B_shunt[b] *1j;

B_line = np.zeros([NN,NN])
B_2line = np.zeros([NN,NN])

for b in range(NB):
    i = int(TO[b])-1
    j = int(FROM[b])-1   
    
    ykm = (r[b]+x[b]*1j)**(-1)
 
    tii[b] = (a[b]*a[b])**(-1)
    tjj[b] = 1
    tij[b] = a[b]**(-1)

    Y[i][i] = Y[i][i] + tii[b]*ykm+ 1j*bsh[b]
    Y[j][j] = Y[j][j]+ tjj[b]*ykm+ 1j*bsh[b]   

    Y[i][j] = Y[i][j] - tij[b]*ykm
    Y[j][i] = Y[j][i] - tij[b]*ykm
    
    B_line[i][i] = B_line[i][i] + 1/x[b]
    B_line[j][j] = B_line[j][j] + 1/x[b]
    
    B_line[i][j] = -1/x[b]
    B_line[j][i] = -1/x[b]
    
    B_2line[i][i] = - 1/x[b]
    
    B_2line[i][j] = -1/x[b]
    B_2line[j][i] = -1/x[b]
    
    if Type[i] == 3 or Type[j] == 3:  
        B_line[i][j] = 0
        B_line[j][i] = 0
        
    if Type[i] ==3:
        B_line[i][i] = 10**10    


G = np.real(Y)               
B = np.imag(Y)            
  
for i in range(NN):
    for j in range(NN):
        if Type[i] < 1 and Type[j] < 1:
            B_2line[i][i] = -B[i][i]
            B_2line[i][j] = -B[i][j]
        if Type[i] > 1 or Type[j] > 1:
            B_2line[i][i] = 0
            B_2line[i][j] = 0
            B_2line[j][i] = 0
    if Type[i] > 1:
        B_2line[i][i] = 10**10   

Initiating the iterative process and reversing the matrices $ B ^{\prime}$ and $ B^{\prime\prime} $.

In [ ]:
B_line_inv= np.linalg.inv(B_line)
B_2line_inv= np.linalg.inv(B_2line)

p = 0
q = 0
KP = 0
KQ = 0

mismP = np.array([])
mismQ = np.array([])

while (p <= 200 or q <= 200):    

    P_Calc = np.zeros_like(P_nom)

    for i in range(NN):
        P_Calc[i] = G[i][i] * V[i]**2
    
    for w in range(NB):    
        i = int(TO[w])-1
        j = int(FROM[w])-1        
    
        ykm = (r[w]+x[w]*1j)**(-1)
        
        g = np.real(ykm) * tij[w]
        b = np.imag(ykm) * tij[w]
    
        ab = theta[i] - theta[j] + phi[w]    
    
        P_Calc[i] = P_Calc[i] + V[i] * V[j] * ( - g * math.cos(ab) - b * math.sin(ab))
        P_Calc[j] = P_Calc[j] + V[i] * V[j] * ( - g * math.cos(ab) + b * math.sin(ab))      

    DP = np.zeros_like(P_nom)    
    
    maxDP = 0   
    
    for i in range(NN):        
        if Type[i]!= 3:
            DP[i] = ((P_nom[i] - P_Calc[i])/V[i])            
            if abs(DP[i]) > abs(maxDP):
                maxDP = DP[i]  

    mismP = np.append(mismP, maxDP)
    
    if abs(maxDP) > zeta:  
        d_theta = np.dot(B_line_inv,DP)
        theta = d_theta + theta          
        KQ = 1
        p += 1
    else:
        KP = 0            
        if KQ == 0:		
            break			
 
    Q_Calc = np.zeros_like(Q_nom)
    
    for i in range(NN):
        Q_Calc[i] = - B[i][i] * V[i]**2
	
    for w in range(NB):    
        i = int(TO[w])-1
        j = int(FROM[w])-1        
    
        ykm = (r[w]+x[w]*1j)**(-1)
        
        g = np.real(ykm) * tij[w]
        b = np.imag(ykm) * tij[w]
    
        ab = theta[i]-theta[j]+ phi[w]  
	
        Q_Calc[i] = Q_Calc[i] + V[i] * V[j] * ( - g * math.sin(ab) + b * math.cos(ab))
        Q_Calc[j] = Q_Calc[j] - V[i] * V[j] * ( - g * math.sin(ab) - b * math.cos(ab))
    
    DQ = np.zeros_like(Q_nom)
    
    maxDQ = 0    

    for i in range(NN):          
        if Type[i]<= 1:
            DQ[i] = ((Q_nom[i] - Q_Calc[i])/V[i]) 
            
            if abs(DQ[i]) > abs(maxDQ):
                maxDQ = DQ[i] 
                
    mismQ = np.append(mismQ, maxDQ)
    
    if abs(maxDQ) > zeta:    
        dV = np.dot(B_2line_inv,DQ)
        V = dV + V    
        KP = 1
        q += 1
    else:
        KQ = 0
        if KP==0:
            break
total_time = time.time() - start_time

Power mismatches graph

In [ ]:
x = range(p)
y1= abs(mismQ)
y2= abs(mismP)
plt.ylabel('Power mismatches')
plt.xlabel('Iterations')

plt.ylim(ymax = 1, ymin = 0)
plt.xlim(xmax = p, xmin = 0)
plt.plot(x, y1, y2)
plt.show()

Voltage profile in the system

In [ ]:
x = range(NN)
y= abs(V)
plt.ylabel('Magnitude of tension [p.u]')
plt.xlabel('Node')

plt.ylim(ymax = 1.2, ymin = 0)
plt.xlim(xmax = 13, xmin = 0)
plt.plot(x, y)
plt.show()